In [ ]:
from lib.data import *
import time
data, raw = get_default_oec_rca_raw_data()

In [ ]:
data

In [ ]:
import economic_complexity as ecplx

rca = ecplx.rca(data)
M = build_M_from(rca)

In [ ]:
#from informe import SimuladorEstatico, SimuladorDinamico, SimuladorComplejo, PaisNaive
from informe import *
from lib.utils import *#correr_simulacion_mostrando, cantidad_descubrimientos_iteracion, cantidad_descubrimientos_paises, pais_collector

De forma similar al trabajo anterior:
https://cnet.fi.uba.ar/netscix23/abstracts/Dynamics%20matter:%20A%20simulation%20framework%20to%20study%20diffusion%20processes%20on%20a%20Dynamic%20Product%20Space.pdf

Vamos a comparar la convergencia tomando dos paises distintos, Argentina y Alemania, según el $\Omega$.

Un **PaisComplejo**, tarda en ser competitivo a un producto alcanzable -según la proximidad- de acuerdo a la complejidad del producto.

De tal manera que al inicio de la simulación se toma el rango de menor a mayor complejidad y se lineariza entre 1 y 10 iteraciones.

**NOTA** explorar distintos calculos de pci -> tiempo?

Como en el paper citado, la convergencia está en el orden de las unidades (3-5 iteraciones aprox) en este trabajo se iterara diez veces esa cantidad.

In [ ]:
cant_paises = len(M.index.to_list())

# 0.1..0.9
def simular_para(sim_clase_estatica= SimuladorComplejoEstatico, pais_clase_estatico = PaisComplejo,
                 sim_clase_dinamica=SimuladorComplejo, pais_clase_dinamico = PaisComplejo,
                 omegas = [], iteraciones=15, mostrar=False):
    for omega in omegas:
        estatico = sim_clase_estatica(lambda step: step >= iteraciones, pais_clase_estatico, M.copy(deep=True), omega)
        dinamico = sim_clase_dinamica(lambda step: step >= iteraciones, pais_clase_dinamico, M.copy(deep=True), omega)
        start = time.time()
        historia_estatico = correr_simulacion_mostrando(estatico, mostrar, 
                                                        collector=default_collector, 
                                                        formatter=default_formatter)
        historia_dinamico = correr_simulacion_mostrando(dinamico, mostrar)
        print(f"elapsed: {time.time() - start} segs")
        yield (historia_estatico, historia_dinamico, omega)

In [ ]:
resultados_paper = list(simular_para(
    sim_clase_dinamica=SimuladorDinamico, sim_clase_estatica=SimuladorEstatico,
    pais_clase_dinamico=PaisNaive, pais_clase_estatico=PaisNaive,
    omegas=[x/10.0 for x in range(1,10)], iteraciones=5))

In [ ]:
resultados_nuevos = list(simular_para(
    sim_clase_dinamica=SimuladorComplejo, sim_clase_estatica=SimuladorComplejoEstatico,
    pais_clase_dinamico=PaisComplejo, pais_clase_estatico=PaisComplejo,
    omegas=[x/10.0 for x in range(1,10)], 
    iteraciones=35))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator

def plot_sps_vs_dps(spd_hist, dps_hist, omega):
    cant_des_sps = cantidad_descubrimientos_iteracion(spd_hist)
    cant_des_sps_arg = cantidad_descubrimientos_iteracion(spd_hist, ["Argentina"])
    cant_des_sps_ger = cantidad_descubrimientos_iteracion(spd_hist, ["Germany"])
    
    cant_des_dps = cantidad_descubrimientos_iteracion(dps_hist)
    cant_des_dps_arg = cantidad_descubrimientos_iteracion(dps_hist, ["Argentina"])
    cant_des_dps_ger = cantidad_descubrimientos_iteracion(dps_hist, ["Germany"])
    
    fig = plt.figure(figsize=(10, 6))
    fig.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.plot(range(len(spd_hist)), cant_des_sps / cant_paises, label='Promedio Mundial - SPS', 
            color = "g", linestyle='dashdot')
    plt.plot(range(len(dps_hist)),cant_des_dps / cant_paises, label='Promedio Mundial - DPS', 
            color = "g", linestyle='-')
    plt.plot(range(len(spd_hist)), cant_des_sps_arg, label='Argentina - SPS', 
            color = "r", linestyle='dashdot')
    plt.plot(range(len(dps_hist)),cant_des_dps_arg, label='Argentina - DPS', 
            color = "r", linestyle='-')
    plt.plot(range(len(spd_hist)), cant_des_sps_ger, label='Alemania - SPS', 
            color = "b", linestyle='dashdot')
    plt.plot(range(len(dps_hist)),cant_des_dps_ger, label='Alemania - DPS', 
            color = "b", linestyle='-')
    plt.xlabel('Iteracion')
    plt.ylabel('SPS <- Diferencia de cantidad de productos exportados -> DPS')
    plt.title(f'Productos exportados Dinámico (DPS) vs Estático (SPS) con Omega = {omega}')
    plt.legend()
    plt.show()

In [ ]:
for (spd_hist, dps_hist, omega) in resultados_paper:
    plot_sps_vs_dps(spd_hist, dps_hist, omega)

In [ ]:
from matplotlib.ticker import FormatStrFormatter

def plot_dps_vs_sps_boxplot(paises, resultados, aclaracion = "", max_it=None):
    prom_diffs = {}
    for (sps_hist, dps_hist, omega) in resultados:
        mit = max_it if max_it else len(sps_hist)
        sps_desc_por_pais = {p: cantidad_descubrimientos_paises(sps_hist[0:mit])[p] for p in paises}
        dps_desc_por_pais = {p: cantidad_descubrimientos_paises(dps_hist[0:mit])[p] for p in paises}
        
        prom_diffs[omega] = np.array(list(dps_desc_por_pais.values()))
        prom_diffs[omega] -= np.array(list(sps_desc_por_pais.values()))
        prom_diffs[omega] = prom_diffs[omega].astype(float)
    fig = plt.figure(figsize=(10, 6))
    ax = plt.boxplot(prom_diffs.values()) #, tick_labels=list(prom_diffs.keys()))
    fig.gca().set_xticklabels(list(prom_diffs.keys()))
    plt.xlabel('Omega')
    plt.ylabel('Promedio de diferencia de exportaciones')
    plt.title(f'Productos extra exportados con DPS en {mit} iteraciones - {aclaracion}')
    plt.show()

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_paper, f"- comportamiento del paper")

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_nuevos, f"comportamiento del sistema complejo", max_it=15)

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_nuevos, f"- comportamiento del sistema complejo", max_it=20)

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_nuevos, f"- comportamiento del sistema complejo", max_it=25)

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_nuevos, f"- comportamiento del sistema complejo", max_it=30)

In [ ]:
plot_dps_vs_sps_boxplot(list(M.index), resultados_nuevos, f"- comportamiento del sistema complejo", max_it=35)

# usando este $\Omega$ graficamos:

In [ ]:
resultados_nuevo

#TODO: Elegir uno de los $\Omega$ para mostrar diferencia entre Arg y Alemania

# TODO: enfocarse solo en las últimas iteraciones; hacer lo mismo para cigarras y hormigas

In [ ]:
plot_dps_vs_sps_boxplot(["Argentina", "Germany"], resultados_paper, "- comportamiento del paper")

In [ ]:
plot_dps_vs_sps_boxplot(["Argentina", "Germany"], resultados_nuevos, 
                        "- comportamiento del sistema complejo (Solo Arg - Ger)")